In [3]:
## notebooks/recommendation_algorithms.ipynb

### Projet RecommandationEngineTG : Algorithmes de Recommandation

# Ce notebook contient l'implémentation et l'explication des algorithmes de recommandation
# pour le projet RecommandationEngineTG.


### 1. Préparation des Données

# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Chargement des jeux de données simulés
# Assurez-vous que les fichiers 'items.csv' et 'users.csv' sont dans le dossier 'data/'
# par rapport à l'emplacement où vous lancez ce notebook.

try:
    items_df = pd.read_csv('../data/items.csv')
    users_df = pd.read_csv('../data/users.csv')
    print("Données chargées avec succès !")
    print("\nItems :")
    print(items_df.head())
    print("\nUtilisateurs (likes) :")
    print(users_df.head())
except FileNotFoundError:
    print("Erreur : Assurez-vous que les fichiers items.csv et users.csv sont bien dans le dossier '../data/'.")
    print("Exemple de structure de dossier :")
    print("RecommandationEngineTG/")
    print("├── data/")
    print("│   ├── users.csv")
    print("│   └── items.csv")
    print("├── notebooks/")
    print("│   └── recommendation_algorithms.ipynb")
    # Création de DataFrames de démonstration si les fichiers ne sont pas trouvés pour permettre la suite du notebook
    items_df = pd.DataFrame({
        'item_id': [101, 102, 103, 104, 105],
        'title': ['Film A', 'Film B', 'Film C', 'Film D', 'Film E'],
        'genre': ['Action', 'Sci-Fi', 'Action', 'Romance', 'Sci-Fi']
    })
    users_df = pd.DataFrame({
        'user_id': [1, 1, 2, 2, 3],
        'item_id': [101, 102, 101, 103, 104],
        'liked': [1, 1, 1, 1, 1]
    })
    print("\nUtilisation de données de démonstration pour le reste du notebook.")


# Création de la matrice de user-item (matrice d'appréciation)
# Pivotons le DataFrame users_df pour avoir les utilisateurs en index, les items en colonnes et les "likes" comme valeurs.
user_item_matrix = users_df.pivot_table(index='user_id', columns='item_id', values='liked').fillna(0)
print("\nMatrice User-Item (extrait) :")
print(user_item_matrix.head())

Données chargées avec succès !

Items :
   item_id            title            genre
0      101        Inception  Science-Fiction
1      102     Interstellar  Science-Fiction
2      103  The Dark Knight           Action
3      104          Titanic          Romance
4      105           Avatar  Science-Fiction

Utilisateurs (likes) :
   user_id  item_id  liked
0        1      101      1
1        1      102      1
2        1      105      1
3        2      101      1
4        2      103      1

Matrice User-Item (extrait) :
item_id  101  102  103  104  105  106  107  108  109  110  111  112  113  114  \
user_id                                                                         
1        1.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2        1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
3        0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
4        0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0 

In [4]:
# ---

### 2. Algorithme de Filtrage Collaboratif Basé sur les Utilisateurs

# L'idée est de trouver des utilisateurs qui ont des préférences similaires à celles de l'utilisateur cible,
# puis de recommander les items que ces "voisins" ont aimés mais que l'utilisateur cible n'a pas encore vus.

def get_user_similarities(user_item_matrix):
    """
    Calcule la similarité cosinus entre tous les utilisateurs.
    Retourne un DataFrame de similarité user-user.
    """
    user_similarity = cosine_similarity(user_item_matrix)
    user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
    return user_similarity_df

def recommend_by_user_similarity(target_user_id, user_item_matrix, user_similarity_df, items_df, num_recommendations=5):
    """
    Recommande des items à un utilisateur cible en utilisant le filtrage collaboratif.
    """
    if target_user_id not in user_item_matrix.index:
        return f"Utilisateur {target_user_id} non trouvé."

    # Obtenir les utilisateurs similaires, exclure l'utilisateur cible lui-même
    similar_users = user_similarity_df[target_user_id].sort_values(ascending=False)
    similar_users = similar_users.drop(target_user_id) # Exclure l'utilisateur lui-même

    # Identifier les items déjà aimés par l'utilisateur cible
    user_liked_items = user_item_matrix.loc[target_user_id][user_item_matrix.loc[target_user_id] == 1].index

    recommendations = []
    # Parcourir les utilisateurs similaires
    for similar_user_id, similarity_score in similar_users.items():
        if similarity_score > 0: # Ne considérer que les utilisateurs avec une similarité positive
            # Obtenir les items aimés par l'utilisateur similaire
            similar_user_liked_items = user_item_matrix.loc[similar_user_id][user_item_matrix.loc[similar_user_id] == 1].index

            # Recommander les items que l'utilisateur similaire a aimés et que l'utilisateur cible n'a pas encore vus
            new_recommendations = [item for item in similar_user_liked_items if item not in user_liked_items]

            # Ajouter les recommandations avec le titre et le genre
            for item_id in new_recommendations:
                item_info = items_df[items_df['item_id'] == item_id].iloc[0]
                recommendations.append({
                    'item_id': item_id,
                    'title': item_info['title'],
                    'genre': item_info['genre'],
                    'reason': f"Aimé par un utilisateur similaire (ID: {similar_user_id}, Similarité: {similarity_score:.2f})"
                })
        if len(recommendations) >= num_recommendations * 2: # Prendre plus pour filtrer ensuite les doublons
            break

    # Supprimer les doublons et limiter le nombre de recommandations
    unique_recommendations = pd.DataFrame(recommendations).drop_duplicates(subset=['item_id']).head(num_recommendations)
    return unique_recommendations.to_dict(orient='records')

In [5]:
# ---
# Exemple d'utilisation du filtrage collaboratif
user_similarity_df = get_user_similarities(user_item_matrix)
print("\nMatrice de Similarité User-User (extrait) :")
print(user_similarity_df.head())

target_user_id = 1
collaborative_recs = recommend_by_user_similarity(target_user_id, user_item_matrix, user_similarity_df, items_df)
print(f"\nRecommandations collaboratives pour l'utilisateur {target_user_id} :")
for rec in collaborative_recs:
    print(f"- {rec['title']} ({rec['genre']}) - Raison: {rec['reason']}")

target_user_id = 4
collaborative_recs = recommend_by_user_similarity(target_user_id, user_item_matrix, user_similarity_df, items_df)
print(f"\nRecommandations collaboratives pour l'utilisateur {target_user_id} :")
for rec in collaborative_recs:
    print(f"- {rec['title']} ({rec['genre']}) - Raison: {rec['reason']}")


Matrice de Similarité User-User (extrait) :
user_id        1         2    3    4         5         6    7    8    9   \
user_id                                                                    
1        1.000000  0.333333  0.0  0.0  0.666667  0.000000  0.0  0.0  0.0   
2        0.333333  1.000000  0.0  0.0  0.000000  0.666667  0.0  0.0  0.0   
3        0.000000  0.000000  1.0  0.0  0.000000  0.000000  0.0  1.0  0.0   
4        0.000000  0.000000  0.0  1.0  0.000000  0.000000  0.0  0.0  1.0   
5        0.666667  0.000000  0.0  0.0  1.000000  0.000000  0.0  0.0  0.0   

user_id        10  
user_id            
1        0.666667  
2        0.666667  
3        0.000000  
4        0.000000  
5        0.333333  

Recommandations collaboratives pour l'utilisateur 1 :
- Blade Runner 2049 (Science-Fiction) - Raison: Aimé par un utilisateur similaire (ID: 5, Similarité: 0.67)
- The Dark Knight (Action) - Raison: Aimé par un utilisateur similaire (ID: 10, Similarité: 0.67)
- Gladiator (Action) 

In [6]:
# ---

### 3. Algorithme de Filtrage Basé sur le Contenu

# L'idée est de recommander des items ayant des caractéristiques (genres) similaires
# à celles des items que l'utilisateur a déjà aimés.

def recommend_by_content(target_user_id, users_df, items_df, num_recommendations=5):
    """
    Recommande des items à un utilisateur cible en utilisant le filtrage basé sur le contenu (genres).
    """
    if target_user_id not in users_df['user_id'].values:
        return f"Utilisateur {target_user_id} non trouvé."

    # Items que l'utilisateur cible a aimés
    user_liked_items_ids = users_df[(users_df['user_id'] == target_user_id) & (users_df['liked'] == 1)]['item_id'].tolist()

    if not user_liked_items_ids:
        return [] # Pas de likes enregistrés pour cet utilisateur

    # Genres préférés de l'utilisateur (basé sur ses items aimés)
    liked_items_info = items_df[items_df['item_id'].isin(user_liked_items_ids)]
    preferred_genres = liked_items_info['genre'].unique()

    # Tous les items qui n'ont pas encore été aimés par l'utilisateur
    unliked_items = items_df[~items_df['item_id'].isin(user_liked_items_ids)]

    recommendations = []
    # Parcourir les genres préférés et trouver des items non vus
    for genre in preferred_genres:
        genre_items = unliked_items[unliked_items['genre'] == genre]
        for _, row in genre_items.iterrows():
            if len(recommendations) < num_recommendations:
                recommendations.append({
                    'item_id': row['item_id'],
                    'title': row['title'],
                    'genre': row['genre'],
                    'reason': f"Similaire à vos préférences de genre ({genre})"
                })
            else:
                break
        if len(recommendations) >= num_recommendations:
            break

    return recommendations

In [7]:
# ---
# Exemple d'utilisation du filtrage basé sur le contenu
target_user_id = 1
content_recs = recommend_by_content(target_user_id, users_df, items_df)
print(f"\nRecommandations basées sur le contenu pour l'utilisateur {target_user_id} :")
for rec in content_recs:
    print(f"- {rec['title']} ({rec['genre']}) - Raison: {rec['reason']}")

target_user_id = 4
content_recs = recommend_by_content(target_user_id, users_df, items_df)
print(f"\nRecommandations basées sur le contenu pour l'utilisateur {target_user_id} :")
for rec in content_recs:
    print(f"- {rec['title']} ({rec['genre']}) - Raison: {rec['reason']}")


Recommandations basées sur le contenu pour l'utilisateur 1 :
- Blade Runner 2049 (Science-Fiction) - Raison: Similaire à vos préférences de genre (Science-Fiction)

Recommandations basées sur le contenu pour l'utilisateur 4 :
- Forrest Gump (Drame) - Raison: Similaire à vos préférences de genre (Drame)
- The Shawshank Redemption (Drame) - Raison: Similaire à vos préférences de genre (Drame)


In [8]:
# ---

### 4. Combinaison des Recommandations

# Il est souvent préférable de combiner les résultats de différentes méthodes pour obtenir
# des recommandations plus robustes et diversifiées.

def get_combined_recommendations(target_user_id, user_item_matrix, user_similarity_df, items_df, users_df, num_recommendations_total=5):
    """
    Combine les recommandations du filtrage collaboratif et du filtrage basé sur le contenu.
    """

    # Récupérer les recommandations collaboratives
    collaborative_recommendations = recommend_by_user_similarity(target_user_id, user_item_matrix, user_similarity_df, items_df, num_recommendations=num_recommendations_total * 2)
    if isinstance(collaborative_recommendations, str): # Gérer le cas où l'utilisateur n'est pas trouvé
        collaborative_recommendations = []

    # Récupérer les recommandations basées sur le contenu
    content_based_recommendations = recommend_by_content(target_user_id, users_df, items_df, num_recommendations=num_recommendations_total * 2)
    if isinstance(content_based_recommendations, str): # Gérer le cas où l'utilisateur n'est pas trouvé
        content_based_recommendations = []

    # Combiner et dédoublonner les recommandations
    combined_recs = []
    seen_item_ids = set()

    # Ajouter les recommandations collaboratives en premier
    for rec in collaborative_recommendations:
        if rec['item_id'] not in seen_item_ids:
            combined_recs.append(rec)
            seen_item_ids.add(rec['item_id'])

    # Ajouter les recommandations basées sur le contenu, en évitant les doublons
    for rec in content_based_recommendations:
        if rec['item_id'] not in seen_item_ids:
            combined_recs.append(rec)
            seen_item_ids.add(rec['item_id'])

    # S'assurer que l'utilisateur n'a pas déjà aimé ces items
    user_liked_items_ids = users_df[(users_df['user_id'] == target_user_id) & (users_df['liked'] == 1)]['item_id'].tolist()
    final_recommendations = [rec for rec in combined_recs if rec['item_id'] not in user_liked_items_ids]

    return final_recommendations[:num_recommendations_total]

In [9]:
# ---
# Exemple d'utilisation des recommandations combinées
user_similarity_df = get_user_similarities(user_item_matrix)

target_user_id = 1
final_recs = get_combined_recommendations(target_user_id, user_item_matrix, user_similarity_df, items_df, users_df)
print(f"\nRecommandations combinées pour l'utilisateur {target_user_id} :")
if final_recs:
    for rec in final_recs:
        print(f"- {rec['title']} ({rec['genre']}) - Raison: {rec['reason']}")
else:
    print("Aucune recommandation disponible.")

target_user_id = 7
final_recs = get_combined_recommendations(target_user_id, user_item_matrix, user_similarity_df, items_df, users_df)
print(f"\nRecommandations combinées pour l'utilisateur {target_user_id} :")
if final_recs:
    for rec in final_recs:
        print(f"- {rec['title']} ({rec['genre']}) - Raison: {rec['reason']}")
else:
    print("Aucune recommandation disponible.")

# ---
# Ce notebook contient les fondations de votre système de recommandation.
# Les fonctions définies ici peuvent être réutilisées dans l'API FastAPI.


Recommandations combinées pour l'utilisateur 1 :
- Blade Runner 2049 (Science-Fiction) - Raison: Aimé par un utilisateur similaire (ID: 5, Similarité: 0.67)
- The Dark Knight (Action) - Raison: Aimé par un utilisateur similaire (ID: 10, Similarité: 0.67)
- Gladiator (Action) - Raison: Aimé par un utilisateur similaire (ID: 2, Similarité: 0.33)

Recommandations combinées pour l'utilisateur 7 :
Aucune recommandation disponible.
